#### Connect to the database, create table if it doesn't exist and extract the number of row

In [8]:
write_to_file(log_file, f"Start extration of {manga_name}","w")

write_to_file(log_file, f"Connecting to MySQL {database} database")

# Connect to the database
try:

    cnx = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        port=3306
    )

    write_to_file(log_file, f"Connected to {database} database")

    # Create a cursor object
    cursor = cnx.cursor()

except mysql.connector.Error as err:
    write_to_file(log_file, f"There is an error : {err}")
    # Close the database cursor and connection
    if cursor:
        cursor.close()
    if cnx:
        cnx.close()
    raise

In [24]:
try:
    # Create a cursor object
    cursor = cnx.cursor()

    # SQL statement to check the tables in the database
    sql = '''Show tables'''

    # Execute the SQL statement
    cursor.execute(sql)

    tables = []
    # Extract table names from the database into a table
    for i in cursor.fetchall():
        for j in i:
            tables.append(i[0])

    # If table doesn't exist into the database, create the table else extract the number of episod already in the table        
    if table_name.lower() in tables:
        write_to_file(log_file, f"Table {table_name} already exist into {database} database")
        write_to_file(log_file, f"Extracting number of episod already in {table_name} table")

        # Extract number of episod in database
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        number_ep_database = cursor.fetchall()
        number_ep_database = int(number_ep_database[0][0])
        write_to_file(log_file, f"There is(are) {number_ep_database} episod(s) already in {table_name} table")

    else:
        number_ep_database = 0
        write_to_file(log_file, f"Creating table {table_name} into {database} database")

        # Create the table
        sql = '''CREATE TABLE IF NOT EXISTS ''' + table_name + ''' (
            Episod INT UNIQUE NOT NULL PRIMARY KEY,
            Season INT NOT NULL,
            Episod_season INT NOT NULL,
            Title VARCHAR(255),
            Day INT,
            Month INT,
            Year INT NOT NULL,
            Rate DOUBLE,
            Link VARCHAR(255)
                )
            '''
        # Execute the SQL statement
        cursor.execute(sql)

        # Commit the changes to the database
        cnx.commit()

        write_to_file(log_file, f"Table {table_name} created into {database} database successfully")

except mysql.connector.Error as err:
    write_to_file(log_file, f"There is an error : {err}")
    # Close the database cursor and connection
    if cursor:
        cursor.close()
    if cnx:
        cnx.close()
    raise     

In [5]:
# Number total of episod
write_to_file(log_file, f"There are {number_tot_ep} {manga_name} episods")

# Calcul number of episod to extract
number_ep_to_extract = number_tot_ep - number_ep_database
write_to_file(log_file, f"There is(are) {number_ep_to_extract} episod(s) to extract")